In [ ]:
# Import necessary library packages

import cv2 
import keras
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pandas_profiling as pp
import tensorflow as tf
import time

from skimage.io import imread
from sklearn import metrics, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Convolution2D, GlobalAveragePooling2D, Activation, Dense, BatchNormalization, Dropout, MaxPooling2D, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.python.keras.engine.base_layer import Layer

# Severstal Steel Dataset

**1. Data Visualisation**

In [ ]:
# Define directories for training and test images
# Display the training and sample submission CSV file

train_dir = '/kaggle/input/severstal-steel-defect-detection/train_images'
test_dir = '/kaggle/input/severstal-steel-defect-detection/test_images'
train_df = pd.read_csv('/kaggle/input/severstal-steel-defect-detection/train.csv')
submission_df = pd.read_csv('/kaggle/input/severstal-steel-defect-detection/sample_submission.csv')

In [ ]:
# Display dataframe for training images

# pd.set_option('display.max_rows', None)
train_df.head(len(train_df))

In [ ]:
# Generate a profile report of the training dataset

pp.ProfileReport(train_df)

In [ ]:
# Summary of the training dataset

train_df.info()

In [ ]:
# Display dataframe for testing images

# pd.set_option('display.max_rows', None)
submission_df.head(len(submission_df))

In [ ]:
# Read a sample training image with its information displayed

display(train_df.iloc[1111])
image = imread(train_dir + '/' + train_df['ImageId'][1111])
# image = imread(train_dir + '/' + '282cd397d.jpg')

#The size of original image is (256, 1600)

plt.figure(figsize = None) 
plt.imshow(image)
plt.show()

In [ ]:
# Display the dimension information for a sample training image

input_shape = image.shape
input_shape

In [ ]:
# Read multiple samples of training images

training = []
training.append(imread(train_dir + '/' + train_df['ImageId'][1111]))
training.append(imread(train_dir + '/' + train_df['ImageId'][2222]))
training.append(imread(train_dir + '/' + train_df['ImageId'][3333]))
training.append(imread(train_dir + '/' + train_df['ImageId'][4444]))
training.append(imread(train_dir + '/' + train_df['ImageId'][5555]))

labels = [] 
labels.append(train_df['ClassId'][1111])
labels.append(train_df['ClassId'][2222])
labels.append(train_df['ClassId'][3333])
labels.append(train_df['ClassId'][4444])
labels.append(train_df['ClassId'][5555])

# Resized training images

plt.figure(figsize = [25,4]) 

for x in range(0,5):
    plt.subplot(3, 3,x + 1)
    plt.subplots_adjust(hspace = 1.0)
    plt.imshow(training[x])
    plt.title("Defect Class: {}".format(labels[x]))
    x += 1
    
plt.show()

In [ ]:
# Creating the dict with classId and Encoded pixels and group all together
# Creating the dict
# train_df['ClassId_EncodedPixels'] = train_df.apply(lambda row: [row['ClassId'], row['EncodedPixels']], axis = 1)

# Grouping together
# grouped_EncodedPixels = train_df.groupby('ImageId')['ClassId_EncodedPixels'].apply(list)

# Display the respective information of the training dataset
print('Number of unique images: %s' % len(train_df['ImageId'].drop_duplicates()))
print('Number of images having at least one defect: %s' % len(train_df[train_df['EncodedPixels'] != -1]['ImageId'].unique()))
print('Total training samples: %s' % len(train_df))

In [ ]:
# Plot the bar graph the number of training images for each class

train_df["ClassId"].value_counts().plot(kind = 'barh')

In [ ]:
# Obtain the number of training images for each class

class_3, class_1, class_4, class_2 = train_df['ClassId'].value_counts()
print("Number of Class 1 defects: ", class_1)
print("Number of Class 2 defects: ", class_2)
print("Number of Class 3 defects: ", class_3)
print("Number of Class 4 defects: ", class_4)

**2. Train-test Split**

In [ ]:
# Function for getting the training ImageId in array form

def get_ImageArray():
    
    # Looping counter
    image_index = 0
    
    # Obtain the total number of images in training set
    total_image_rows = len(train_df['ImageId'])
    
    # Create an empty array named labels for inserting the ImageId
    images = []
    
    # Append each image into the images array and iterate until the total number of training images
    for image_index in range(total_image_rows):
        img_arr = cv2.imread(train_dir + '/' + train_df['ImageId'][image_index])
        img_arr = cv2.resize(img_arr, (120,120))
        images.append(img_arr)
        image_index += 1
        
    # Return the labels as the function output
    return images

In [ ]:
# Assigning the output of get_ImageArray() function to variable images

images = get_ImageArray()

In [ ]:
# Function for getting the ClassId in array form

def get_ClassId():
    
    # Looping counter
    class_index = 0
    
    # Obtain the total number of images in training set
    total_class_rows = len(train_df['ClassId'])
    
    # Create an empty array named labels for inserting the ClassId
    class_labels = []
    
    # Append each label into the labels array and iterate until the total number of training images
    for class_index in range(total_class_rows):
        class_labels.append(train_df['ClassId'][class_index])
        class_index += 1
        
    # Return the labels as the function output
    return class_labels

In [ ]:
# Assigning the output of get_ClassId() function to variable class_labels

class_labels = get_ClassId()

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

encoder = LabelEncoder()

X = np.array(images)
X = X/255

Y = encoder.fit_transform(class_labels)
Y = to_categorical(Y)

In [ ]:
# Divide the training images into training set and test set

X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, test_size = 0.15, shuffle = True)

In [ ]:
# Obtain the splitting results for the training set and test set

print("x_train shape:", X_train.shape)
print("x_valid shape:", X_valid.shape)
print("y_train shape:", Y_train.shape)
print("y_valid shape:", Y_valid.shape)

**3. Methodology 1 - EffcientNetV2**

In [ ]:
# https://colab.research.google.com/github/google/automl/blob/master/efficientnetv2/tfhub.ipynb#scrollTo=9f3yBUvkd_VJ

import itertools
import os

import matplotlib.pylab as plt
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub

print('TF version:', tf.__version__)
print('Hub version:', hub.__version__)
print('Physical devices:', tf.config.list_physical_devices())

def get_hub_url_and_isize(model_name, ckpt_type, hub_type):
  if ckpt_type == '1k':
    ckpt_type = ''  # json doesn't support empty string
  else:
    ckpt_type = '-' + ckpt_type  # add '-' as prefix
  
  hub_url_map = {
    'efficientnetv2-b0': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b0/{hub_type}',
    'efficientnetv2-b1': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b1/{hub_type}',
    'efficientnetv2-b2': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b2/{hub_type}',
    'efficientnetv2-b3': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b3/{hub_type}',
    'efficientnetv2-s':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-s/{hub_type}',
    'efficientnetv2-m':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-m/{hub_type}',
    'efficientnetv2-l':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-l/{hub_type}',

    'efficientnetv2-b0-21k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b0-21k/{hub_type}',
    'efficientnetv2-b1-21k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b1-21k/{hub_type}',
    'efficientnetv2-b2-21k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b2-21k/{hub_type}',
    'efficientnetv2-b3-21k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b3-21k/{hub_type}',
    'efficientnetv2-s-21k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-s-21k/{hub_type}',
    'efficientnetv2-m-21k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-m-21k/{hub_type}',
    'efficientnetv2-l-21k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-l-21k/{hub_type}',
    'efficientnetv2-xl-21k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-xl-21k/{hub_type}',

    'efficientnetv2-b0-21k-ft1k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b0-21k-ft1k/{hub_type}',
    'efficientnetv2-b1-21k-ft1k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b1-21k-ft1k/{hub_type}',
    'efficientnetv2-b2-21k-ft1k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b2-21k-ft1k/{hub_type}',
    'efficientnetv2-b3-21k-ft1k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b3-21k-ft1k/{hub_type}',
    'efficientnetv2-s-21k-ft1k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-s-21k-ft1k/{hub_type}',
    'efficientnetv2-m-21k-ft1k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-m-21k-ft1k/{hub_type}',
    'efficientnetv2-l-21k-ft1k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-l-21k-ft1k/{hub_type}',
    'efficientnetv2-xl-21k-ft1k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-xl-21k-ft1k/{hub_type}',
      
    # efficientnetv1
    'efficientnet_b0': f'https://tfhub.dev/tensorflow/efficientnet/b0/{hub_type}/1',
    'efficientnet_b1': f'https://tfhub.dev/tensorflow/efficientnet/b1/{hub_type}/1',
    'efficientnet_b2': f'https://tfhub.dev/tensorflow/efficientnet/b2/{hub_type}/1',
    'efficientnet_b3': f'https://tfhub.dev/tensorflow/efficientnet/b3/{hub_type}/1',
    'efficientnet_b4': f'https://tfhub.dev/tensorflow/efficientnet/b4/{hub_type}/1',
    'efficientnet_b5': f'https://tfhub.dev/tensorflow/efficientnet/b5/{hub_type}/1',
    'efficientnet_b6': f'https://tfhub.dev/tensorflow/efficientnet/b6/{hub_type}/1',
    'efficientnet_b7': f'https://tfhub.dev/tensorflow/efficientnet/b7/{hub_type}/1',
  }
  
  image_size_map = {
    'efficientnetv2-b0': 224,
    'efficientnetv2-b1': 240,
    'efficientnetv2-b2': 260,
    'efficientnetv2-b3': 300,
    'efficientnetv2-s':  384,
    'efficientnetv2-m':  480,
    'efficientnetv2-l':  480,
    'efficientnetv2-xl':  512,
  
    'efficientnet_b0': 224,
    'efficientnet_b1': 240,
    'efficientnet_b2': 260,
    'efficientnet_b3': 300,
    'efficientnet_b4': 380,
    'efficientnet_b5': 456,
    'efficientnet_b6': 528,
    'efficientnet_b7': 600,
  }
  
  hub_url = hub_url_map.get(model_name + ckpt_type)
  image_size = image_size_map.get(model_name, 224)
  return hub_url, image_size


def get_imagenet_labels(filename):
  labels = []
  with open(filename, 'r') as f:
    for line in f:
      labels.append(line.split('\t')[1][:-1])  # split and remove line break.
  return labels

In [ ]:
hub_url = 'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b0/feature-vector'
do_fine_tuning = True
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape = [224, 224, 3]),
    hub.KerasLayer(hub_url, trainable = do_fine_tuning),
    tf.keras.layers.Dropout(rate = 0.2),
    tf.keras.layers.Dense(4, activation = 'softmax'),
])

In [ ]:
model.summary()

In [ ]:
model.compile(
  optimizer=tf.keras.optimizers.SGD(learning_rate=0.005, momentum=0.9), 
  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
  metrics=['accuracy'])

In [ ]:
steps_per_epoch = 189
validation_steps = 34

In [ ]:
num_epochs = 25 #@param {type:"integer"}

In [ ]:
history = model.fit(X_train, Y_train, epochs = num_epochs, steps_per_epoch = steps_per_epoch, validation_data = (X_valid, Y_valid), validation_steps = validation_steps, batch_size = 32, verbose = 1).history

In [ ]:
# Plot the graphs for the metrics of interest

acc = history['accuracy']
val_acc = history['val_accuracy']
loss = history['loss']
val_loss = history['val_loss']

epochs = range(1,len(acc) + 1)

plt.plot(epochs, acc,'r',label = 'Training Accuracy')
plt.plot(epochs,val_acc,'b',label = 'Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.figure()

plt.plot(epochs,loss,'r',label = 'Training loss')
plt.plot(epochs,val_loss,'b',label = 'Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.show()

In [ ]:
# https://www.tensorflow.org/api_docs/python/tf/keras/Model
# print(model.metrics_names)
# compute predictions

results = model.evaluate(X_valid, Y_valid)

In [ ]:
# Function for getting the testing ImageId in array form

def get_testImageArray():
    
    # Looping counter
    image_index = 0
    
    # Obtain the total number of images in training set
    total_image_rows = len(submission_df['ImageId'])
    
    # Create an empty array named labels for inserting the ImageId
    test_images = []
    
    # Append each image into the images array and iterate until the total number of training images
    for image_index in range(total_image_rows):
        testimg_arr = cv2.imread(test_dir + '/' + submission_df['ImageId'][image_index])
        testimg_arr = cv2.resize(testimg_arr, (120,120))
        test_images.append(testimg_arr)
        image_index += 1
        
    # Return the labels as the function output
    return test_images

In [ ]:
X_test = get_testImageArray()

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

encoder = LabelEncoder()

X_test = np.array(X_test)
X_test = X_test/255

In [ ]:
pred = model.predict(X_test)
predictions = np.argmax(pred, axis = 1)

In [ ]:
predictions

In [ ]:
classes = ['1','2','3','4']
classes = np.array(classes)
pred_name = classes[predictions]
print(pred_name)

In [ ]:
submission_df['ClassId'] = pred_name

In [ ]:
submission_df.to_csv('./submission.csv', index = False)
submission_df

In [ ]:
# Display dataframe for testing images

pd.set_option('display.max_rows', None)
submission_df.head(len(submission_df))

**4. Run-length Encoding**

In [ ]:
def mask_to_rle(image):
    # The input image is expressed as a numpy array
    # Masked pixels are expressed as value '1' where the background pixels will be set as '0'
    # Returns run length as string formatted
    
    pixels = image.flatten() # Decoded pixels after flattening the masked image
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1 # This record is the position where the bit value starts to change, here +1 is for position adjustment

    runs[1::2] -= runs[::2] # runs[1::2] = runs[1::2] - runs[::2]
    # For example, the result before running is runs = [1 3 13 15] 
    # Each number in runs represents the position where the pixel value changes
    # The result after running is runs = [ 1  2 13  2]
    # Means that starting from the first position, a total of 2 bits are the same, so use 3 - 1 to get 2.
    # Means that starting from the 13th position, a total of 2 bits are the same, so use 15 - 13 to get 2.
    # Correspond to the two 11 at the head and the end above
    
    # Python sequence slice addresses can be written as a[start:end:step] and any of start, stop or end can be dropped. 
    # For example, a[::3] is every third element of the sequence.
    return ' '.join(str(x) for x in runs)
 
def rle_to_mask(mask_rle, shape = (256, 1600)):
    # The mask_rle is the run-length as string formated (start length)
    # image.shape is the dimension of the image in array form
    # Returns a masked image expressed in numpy array, with masked pixels as '1' and background pixels as '0'

    s = mask_rle.split()
    # starts is the list of the position numbers of the changed pixels
    starts, lengths = [np.asarray(x, dtype = int) for x in (s[0:][::2], s[1:][::2])]  
    # s[0:][::2] - This is a list of position numbers of changed pixels, for example, we have encoded pixels as [1, 13]
    # s[1:][::2] - This is a list of the length of the same pixel (recording the consecutive equivalent pixels behind each changed pixel separately Continuous length of value)
    
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype = np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

In [ ]:
EncodedPixels = train_df['EncodedPixels'][22]
EncodedPixels

In [ ]:
print(len(EncodedPixels.split()))
EncodedPixels.split()

In [ ]:
rle = list(map(int, EncodedPixels.split()))
rle

In [ ]:
pixel,pixel_count = [],[]
[pixel.append(rle[i]) if i % 2 == 0 else pixel_count.append(rle[i]) for i in range(0, len(rle))]
print('Pixel starting points:\n',pixel)
print('Pixel counting:\n', pixel_count)

In [ ]:
rle_pixels = [list(range(pixel[i],pixel[i] + pixel_count[i])) for i in range(0, len(pixel))]
print('rle_pixels\n:', rle_pixels)

In [ ]:
rle_mask_pixels = sum(rle_pixels,[]) 
print('rle mask pixels:\n', rle_mask_pixels)

In [ ]:
# Read a sample training image with its information displayed

display(train_df.iloc[22])
image = imread(train_dir + '/' + train_df['ImageId'][22])
print(image.shape)

plt.figure(figsize=None) #The size of original image is (1600, 256)
plt.imshow(image)
plt.show()

In [ ]:
mask_img = np.zeros((256*1600,1), dtype = int)

In [ ]:
mask_img[rle_mask_pixels] = 255

In [ ]:
image = train_dir + '/' + train_df['ImageId'][22]

l,b = cv2.imread(image).shape[0], cv2.imread(image).shape[1]
mask = np.reshape(mask_img, (b, l)).T

In [ ]:
plt.imshow(mask)

In [ ]:
mask_to_rle(rle_to_mask(train_df['EncodedPixels'].iloc[0])) == train_df['EncodedPixels'].iloc[0]

In [ ]:
mask_to_rle(rle_to_mask('1 1')) == '1 1'

In [ ]:
rle_to_mask(train_df['EncodedPixels'].iloc[1111])

In [ ]:
mask_to_rle(rle_to_mask(train_df['EncodedPixels'].iloc[1111]))

# References

1. Main Reference Link 1: https://medium.com/analytics-vidhya/severstal-steel-defect-detection-2d2e836855c2
2. Main Reference link 2: https://medium.com/@saivenkat_/a-detailed-case-study-on-severstal-steel-defect-detection-can-we-detect-and-classify-defects-in-2844402392cc
3. Main Reference Link 3: https://medium.com/@guildbilla/steel-defect-detection-image-segmentation-using-keras-dae8b4f986f0
4. Main Reference Link 4: https://www.kaggle.com/kenmatsu4/visualize-steel-defect
5. Main Reference Link 5: https://www.kaggle.com/yasserhessein/severstal-steel-defect-detection-with-xception
6. Main Reference Link 6: http://faculty.neu.edu.cn/me/songkc/Research.html
7. Main Reference Link 7: https://zhuanlan.zhihu.com/p/92547750
8. Secondary Reference Link 1: https://github.com/nikhilroxtomar/Multiclass-Segmentation-in-Unet
9. Secondary Reference Link 2 (for multiclass classification): https://www.kaggle.com/lsind18/gemstones-multiclass-classification-cnn , https://www.kaggle.com/yemregundogmus/car-forecast-with-multi-class-classification
10. Supportive Reference Link 1: https://www.kaggle.com/arjunrao2000/beginners-guide-efficientnet-with-keras/notebook
11. Supportive Reference Link 2: https://colab.research.google.com/drive/1vzEDAX-3ol7gcZ7qmKuwn8zUld524sUZ#scrollTo=e4wFQJkBiFWE https://blog.roboflow.com/how-to-train-efficientnet/
12. Supportive Reference Link 3: https://towardsdatascience.com/feature-engineering-for-machine-learning-3a5e293a5114
13. Supportive Reference Link 4: https://towardsdatascience.com/building-convolutional-neural-networks-in-python-using-keras-7e4652f6456f

**Image Segmentation** <br>
1. Reference Link 1: https://medium.com/analytics-vidhya/generating-masks-from-encoded-pixels-semantic-segmentation-18635e834ad0 <br> 
2. Reference Link 2: https://www.kaggle.com/paulorzp/run-length-encode-and-decode <br>
3. Reference Link 3: https://www.programmersought.com/article/51974806761/ <br>
4. Reference Link 4: https://developer.nvidia.com/blog/building-image-segmentation-faster-using-jupyter-notebooks-from-ngc/